<a href="https://colab.research.google.com/github/SFStefenon/synthetic_ED/blob/main/c_gan_convert_to_jpg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import cv2
from numpy import asarray
import numpy as np
from PIL import Image

samples = 50
show = False

# Images generated by cGAN
fake_images = torch.load('./saved_results/R_cGAN_fake_images.pt')
i = 0; s_class = 0
for images in fake_images:
    images_a = asarray(images)
    if show == True:
        cv2.imshow("Img", images_a[0]*255)
        cv2.waitKey(500)
    i += 1

    # Save image by image (i) given the its class (s_class)
    filename: str = './inputdata/synthetic_cGAN/' + str(s_class) + '/image_' +  str(s_class) + '_' + str(i) + '.jpg'
    cv2.imwrite(filename, images_a[0]*255)

    img = Image.open(filename)
    # Define a threshold to consider the image as a drawing
    threshold = 100; all_p = []
    for j in range(0, img.size[0]):
        # Where values are lower than the threshold sum it, if they are significant
        if np.sum(np.where(images_a[0][j]*255<threshold))>threshold:
            # Save their horizontal position from min to max values
            horizontal = min(np.where(images_a[0][j]*255<threshold))
            # Save the where it happened on vertical (j), min horizontal, and max horizontal
            position = j, horizontal[0], horizontal[-1]
            all_p.append(position)

    all_pr = np.array(all_p)
    # Define the position of the cropout
    try:
        #bb = (min(all_pr[:,1]), all_p[0][0]-2, max(all_pr[:,2]), all_p[-1][0]+5)
        bb = (min(all_pr[:,1]), all_p[0][0], max(all_pr[:,2]), all_p[-1][0])
        # Cropout the BBs
        image = img.crop(bb)
        imag0 = asarray(image)

        # Save the cropout
        filename2: str = './inputdata/synthetic_cGAN_Crop/' + str(s_class) + '/image_' +  str(s_class) + '_' + str(i) + '.jpg'
        cv2.imwrite(filename2, imag0)
        # print(filename2)
    except: # When there is no class
        pass

    # Start counting
    if i==samples:
        i = 0; s_class += 1

cv2.destroyAllWindows()